In [2]:
print("Hello..")

Hello..


In [19]:
# Import necessary libraries
import sagemaker
from sagemaker import get_execution_role
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import time
from sagemaker.session import Session
from sagemaker import Model
from sagemaker import ModelPackage

# Initialize SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Define the S3 bucket and prefix for storing data
s3_bucket = sagemaker_session.default_bucket()  # Default bucket created by SageMaker
s3_prefix = 'emp-bonus-linearlearner'  # Prefix for organizing data in S3

# This has to be updated based on the previous model training job
training_job_name = 'linear-learner-2025-07-30-01-14-47-200'

print(f"Using bucket: {s3_bucket}")

Using bucket: sagemaker-us-east-1-975050014577


### Step 1: Prepare Model

In [20]:
# Upload the model artifact to S3
local_file_path = 'data/model.tar.gz'
s3_uri = sagemaker_session.upload_data(local_file_path, bucket=s3_bucket, key_prefix = s3_prefix)
print(f"File uploaded to: {s3_uri}")
model_artifact = f's3://{s3_bucket}/{s3_prefix}/model.tar.gz'

# Determine what AWS region we are in
region = sagemaker_session.boto_region_name
# Retrieve the Linear Learner image URI
container_image_uri = sagemaker.image_uris.retrieve(framework="linear-learner", region=region)

print(model_artifact)

# Print path to container image
print(f"SageMaker Linear Learner Image URI: {container_image_uri}")

# Assign a meaningful name to the model
model_name = 'emp-bonus'  # Give it a name for identification

# Create the SageMaker model
model = Model(
    image_uri=container_image_uri,  # Use the appropriate image URI for your model's container
    model_data=model_artifact,      # Path to the model artifact (model.tgz)
    role=get_execution_role(),
    name=model_name
)

File uploaded to: s3://sagemaker-us-east-1-975050014577/emp-bonus-linearlearner/model.tar.gz
s3://sagemaker-us-east-1-975050014577/emp-bonus-linearlearner/model.tar.gz
SageMaker Linear Learner Image URI: 382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:1


### Step 2: Create a Model Package Group in SageMaker Model Registry

In [21]:
# Create a model package group
model_package_group_name = 'emp-bonus-prediction-group'
model_package_group_description = 'SageMaker model package group for employee bonus prediction'
sagemaker_client = boto3.client('sagemaker')

try:
    sagemaker_client.create_model_package_group(
        ModelPackageGroupName=model_package_group_name,
        ModelPackageGroupDescription=model_package_group_description
    )
    print(f'Created Model Package Group: {model_package_group_name}')
    
except sagemaker_client.exceptions.ResourceInUse:
    print(f'Model Package Group {model_package_group_name} already exists.')


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4 sagemaker_client = boto3.client('sagemaker')                                                │
│    5                                                                                             │
│    6 try:                                                                                        │
│ ❱  7 │   sagemaker_client.create_model_package_group(                                            │
│    8 │   │   ModelPackageGroupName=model_package_group_name,                                     │
│    9 │   │   ModelPackageGroupDescription=model_package_group_description                        │
│   10 │   )                                                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ClientError: An error occurred (ValidationException) when calling the CreateModelPackageGroup operation: Model 
Package Group already exists: 
arn:aws:sagemaker:us-east-1:975050014577:model-package-group/emp-bonus-prediction-group

### Step 3: Register Model into SageMaker Model Registry

In [22]:
# Register the model

model_approval_status = "PendingManualApproval"
customer_metadata_properties = {"ModelType": "EmpBonusPrediction"}

print(model_approval_status)
print(model_package_group_name)
print(customer_metadata_properties)
model_package = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    customer_metadata_properties=customer_metadata_properties,
)

print(f"Model package version ARN: {model_package.model_package_arn}")

PendingManualApproval
emp-bonus-prediction-group
{'ModelType': 'EmpBonusPrediction'}


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 print(model_approval_status)                                                                │
│    7 print(model_package_group_name)                                                             │
│    8 print(customer_metadata_properties)                                                         │
│ ❱  9 model_package = model.register(                                                             │
│   10 │   content_types=["text/csv"],                                                             │
│   11 │   response_types=["text/csv"],                                                            │
│   12 │   inference_instances=["ml.m5.large", "ml.m5.large"],                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/model.py:605 in register                       │
│                                                                                                  │
│    602 │   │   │   model_card=model_card,                                                        │
│    603 │   │   │   model_life_cycle=model_life_cycle,                                            │
│    604 │   │   )                                                                                 │
│ ❱  605 │   │   model_package = self.sagemaker_session.create_model_package_from_containers(      │
│    606 │   │   │   **model_pkg_args                                                              │
│    607 │   │   )                                                                                 │
│    608 │   │   if isinstance(self.sagemaker_session, PipelineSession):                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'NoneType' object has no attribute 'create_model_package_from_containers'